# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
import pandas as pd

# Load the dataset
df = pd.read_csv('homepage_actions.csv')

# Check the first 5 rows of the data
df.head()



,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [2]:
# Get the number of unique viewers
num_viewers = df['id'].nunique()
print("Number of unique viewers:", num_viewers)

# Get the number of viewers who also clicked
num_clicks = df.query('action == "click"')['id'].nunique()
print("Number of viewers who also clicked:", num_clicks)



Number of unique viewers: 6328
Number of viewers who also clicked: 1860


In [3]:
# Check for anomalies: viewers who clicked but didn't view
click_view = df.groupby(['id', 'action']).size().reset_index(name='counts')
view_only = click_view[click_view['action'] == 'view']['id'].nunique()
click_only = click_view[click_view['action'] == 'click']['id'].nunique()
print("Number of viewers who only viewed:", view_only)
print("Number of viewers who only clicked:", click_only)

# Check for overlap between control and experiment groups
control = df[df['group'] == 'control']
experiment = df[df['group'] == 'experiment']
overlap = set(control['id']).intersection(set(experiment['id']))
print("Number of viewers who belong to both control and experiment groups:", len(overlap))

# Plan to account for the overlap in the experimental design
# One possible solution is to remove the overlap from both groups and ensure that each 
# unique id belongs to only one group

df = df[~df['id'].isin(overlap)]
df = df.sort_values(by=['timestamp'])
df['id_group'] = df.groupby('id').ngroup()
df['group'] = df['group'] + '_' + df['id_group'].astype(str)


Number of viewers who only viewed: 6328
Number of viewers who only clicked: 1860
Number of viewers who belong to both control and experiment groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
import scipy.stats as stats

df = pd.read_csv('homepage_actions.csv')
df['action'] = df['action'].map({'view': 0, 'click': 1})


In [19]:
# Calculate the average click-through rate for each group
control_mean = df['action'].mean()
experiment_mean = df['action'].mean()

# Calculate the difference in means
diff_mean = experiment_mean - control_mean

# Calculate the standard error of the difference in means
se_diff_mean = ((df['action'].var()/len(control)) + (df['action'].var()/len(df)))**0.5

# Calculate the t-statistic
t_stat = diff_mean / se_diff_mean




In [34]:
# Print the results
print("Difference in means:", diff_mean)
print("Standard error of the difference in means:", se_diff_mean)
print("t-statistic:", t_stat)

Difference in means: 0.0
Standard error of the difference in means: 0.007913351122009112
t-statistic: 0.0


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [32]:
# Calculate the CTR for the control group
control_ctr = df[df['group'] == 'control']['action'].mean()

# Calculate the expected number of clicks for the experiment group if it had the same CTR as the control group
expected_clicks = df[df['group'] == 'experiment'].shape[0] * control_ctr

print('The number of expected clicks for the experiment group if it had the same CTR as the control group is:', expected_clicks)
print('The click-through rate of the control group is:', control_ctr)

import numpy as np

experiment_n = df[df['group'] == 'experiment'].shape[0]
experiment_mean = df[df['group'] == 'experiment']['action'].mean()

standard_deviation = np.sqrt(experiment_n * experiment_mean * (1 - experiment_mean))



The number of expected clicks for the experiment group if it had the same CTR as the control group is: 857.6848030018762
The click-through rate of the control group is: 0.21857410881801126


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [33]:
z_score = (experiment_mean - control_ctr) / standard_deviation

print('The z-score of the experiment group is', z_score)

The z-score of the experiment group is 0.000673193332366104


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [35]:
# Calculate the p-value
p_value = stats.t.sf(abs(t_stat), len(df) + len(df) - 2) * 2

print("p-value:", p_value)

p-value: 1.0


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.